In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis

In [32]:
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    s1                 = solve(ode==0, diff(dependent(independent),independent,independent))
    determining_system = prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    # so far it is checked manually and with mathematica
    determining_system = \
        [_ [0] for _ in determining_system.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    for _ in determining_system:
        _.show()
    janet = Janet_Basis(determining_system, [phi, xi], [dependent, independent])
    return janet

In [21]:
x = var('x')
y = function("y")
ode1=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode1, y ,x)

/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/expression_conversions.py:2347: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self.new(*[self(_) for _ in ex.operands()])
/researches/mambaforge/envs/sage/lib/python3.9/site-packages/sage/symbolic/operators.py:121: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.
  return self._f(*args).diff(*vars)


12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - 24*phi(y(x), x)*y(x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x)

-36*y(x)^3*D[0](xi)(y(x), x) + 8*y(x)*D[0, 1](phi)(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x) - 6*D[1](phi)(y(x), x)

4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) + 3*phi(y(x), x)/y(x) - 3*D[0](phi)(y(x), x)

-4*y(x)*D[0, 0](xi)(y(x), x) - 3*D[0](xi)(y(x), x)

In [22]:
for _ in inf.S:
    _.show()
    _.Lder().show()

-3/4*D[0](phi)(y(x), x)/y(x) + 3/4*phi(y(x), x)/y(x)^2 + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

D[0, 0](phi)(y(x), x)

-9/2*y(x)^2*D[0](xi)(y(x), x) - 3/4*D[1](phi)(y(x), x)/y(x) + D[0, 1](phi)(y(x), x) - 1/2*D[1, 1](xi)(y(x), x)

D[0, 1](phi)(y(x), x)

3*y(x)^2*D[0](phi)(y(x), x) - 6*y(x)^2*D[1](xi)(y(x), x) - 6*phi(y(x), x)*y(x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

3/4*D[0](xi)(y(x), x)/y(x) + D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [26]:
ode2=4*diff(y(x),x,x)*y(x)**2 - 3* (x**2)*diff(y(x),x)*2-12*y(x)**3
inf = infinitesimalsODE(ode2, y ,x)

12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - 12*phi(y(x), x)*y(x)^2 - 6*x^2*D[1](phi)(y(x), x) + 4*y(x)^2*D[1, 1](phi)(y(x), x)

-36*y(x)^3*D[0](xi)(y(x), x) + 8*y(x)^2*D[0, 1](phi)(y(x), x) - 6*x^2*D[1](xi)(y(x), x) - 4*y(x)^2*D[1, 1](xi)(y(x), x) - 12*x*xi(y(x), x) + 12*x^2*phi(y(x), x)/y(x)

4*y(x)^2*D[0, 0](phi)(y(x), x) - 12*x^2*D[0](xi)(y(x), x) - 8*y(x)^2*D[0, 1](xi)(y(x), x)

-4*y(x)^2*D[0, 0](xi)(y(x), x)

AttributeError: 'function' object has no attribute 'function'

In [25]:
for _ in inf.S:
    _.show()
    _.Lder().show()

-3/4*D[0](phi)(y(x), x)/y(x)^2 + 3/2*phi(y(x), x)/y(x)^3 + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

D[0, 0](phi)(y(x), x)

-9/2*y(x)*D[0](xi)(y(x), x) - 3/4*D[1](phi)(y(x), x)/y(x)^2 + D[0, 1](phi)(y(x), x) - 1/2*D[1, 1](xi)(y(x), x)

D[0, 1](phi)(y(x), x)

3*y(x)*D[0](phi)(y(x), x) - 6*y(x)*D[1](xi)(y(x), x) - 3*phi(y(x), x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

3/4*D[0](xi)(y(x), x)/y(x)^2 + D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [30]:
ode2=diff(y(x),x,x) + y(x)*diff(y(x),x)+x*y(x)**4
inf = infinitesimalsODE(ode2, y ,x)

-x*y(x)^4*D[0](phi)(y(x), x) + 2*x*y(x)^4*D[1](xi)(y(x), x) + 4*x*phi(y(x), x)*y(x)^3 + xi(y(x), x)*y(x)^4 + y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

3*x*y(x)^4*D[0](xi)(y(x), x) + y(x)*D[1](xi)(y(x), x) + phi(y(x), x) + 2*D[0, 1](phi)(y(x), x) - D[1, 1](xi)(y(x), x)

2*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

-D[0, 0](xi)(y(x), x)

AttributeError: 'function' object has no attribute 'function'

In [31]:
ode2=diff(y(x),x,x) + 3*y(x)*diff(y(x),x)+y(x)**3
inf = infinitesimalsODE(ode2, y ,x)

-y(x)^3*D[0](phi)(y(x), x) + 2*y(x)^3*D[1](xi)(y(x), x) + 3*phi(y(x), x)*y(x)^2 + 3*y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

3*y(x)^3*D[0](xi)(y(x), x) + 3*y(x)*D[1](xi)(y(x), x) + 3*phi(y(x), x) + 2*D[0, 1](phi)(y(x), x) - D[1, 1](xi)(y(x), x)

6*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

-D[0, 0](xi)(y(x), x)

AttributeError: 'function' object has no attribute 'function'

In [33]:
ode2=diff(y(x),x,x,x) + y(x)*diff(y(x),x,x)
inf = infinitesimalsODE(ode2, y ,x)

y(x)*D[1, 1](phi)(y(x), x) - D[1](xi)(y(x), x)*diff(y(x), x, x, x) - phi(y(x), x)*diff(y(x), x, x, x)/y(x) - 3*D[0, 1](phi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[1, 1](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) - 3*D[0](xi)(y(x), x)*diff(y(x), x, x, x)^2/y(x)^2 + D[1, 1, 1](phi)(y(x), x)

2*y(x)*D[0, 1](phi)(y(x), x) - y(x)*D[1, 1](xi)(y(x), x) - D[0](xi)(y(x), x)*diff(y(x), x, x, x) - 3*D[0, 0](phi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 9*D[0, 1](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[0, 1, 1](phi)(y(x), x) - D[1, 1, 1](xi)(y(x), x)

y(x)*D[0, 0](phi)(y(x), x) - 2*y(x)*D[0, 1](xi)(y(x), x) + 6*D[0, 0](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[0, 0, 1](phi)(y(x), x) - 3*D[0, 1, 1](xi)(y(x), x)

-y(x)*D[0, 0](xi)(y(x), x) + D[0, 0, 0](phi)(y(x), x) - 3*D[0, 0, 1](xi)(y(x), x)

-D[0, 0, 0](xi)(y(x), x)

AttributeError: 'function' object has no attribute 'function'